In [ ]:
# To do:
# Transfer readme to outline/make outline
# Make proj pres outline/find images
# Problem statement
# Start scraping

In [ ]:
# H1 Title 

In [ ]:
# H2 Data Collection, Cleaning, & EDA

In [ ]:
# Picture

In [ ]:
# H4 Problem statement

In [ ]:
# Problem Statement
# - Is it clear what the goal of the project is?
# - What type of model will be developed?
# - How will success be evaluated?
# - Is the scope of the project appropriate?
# - Is it clear who cares about this or why this is important to investigate?
# - Does the student consider the audience and the primary and secondary stakeholders?

In [ ]:
# H4 Data info

In [ ]:
# Data info and background 

In [ ]:
# H4  Subreddit 1 

In [ ]:
# Subreddit 1 info

In [ ]:
# H4 Subreddit 2 

In [ ]:
# Subreddit 2 info

In [ ]:
# Research questions

In [ ]:
# H2 Data Collection
# - Was enough data gathered to generate a significant result?
# - Was data collected that was useful and relevant to the project?
# - Was data collection and storage optimized through custom functions, pipelines, and/or automation?
# - Was thought given to the server receiving the requests such as considering number of requests per second?

In [ ]:
# H4 imports

In [39]:
import requests
import pandas as pd
import json
import csv
import time
import datetime

In [40]:
url = 'https://api.pushshift.io/reddit/search/submission' #?subreddit=mysubreddit'

In [41]:
params = {
'subreddit': 'boardgames',
'size':100}

In [42]:
res = requests.get(url, params)

In [43]:
res.status_code

200

In [44]:
data=res.json()

In [45]:
posts = data['data']

In [46]:
len(posts)

100

In [47]:
import pandas as pd

In [48]:
df = pd.DataFrame(posts)

In [49]:
df.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_is_blocked',
       'author_patreon_flair', 'author_premium', 'awarders', 'can_mod_post',
       'contest_mode', 'created_utc', 'domain', 'full_link', 'gildings', 'id',
       'is_created_from_ads_ui', 'is_crosspostable', 'is_meta',
       'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable',
       'is_self', 'is_video', 'link_flair_background_color',
       'link_flair_richtext', 'link_flair_text_color', 'link_flair_type',
       'locked', 'media_only', 'no_follow', 'num_comments', 'num_crossposts',
       'over_18', 'parent_whitelist_status', 'permalink', 'pinned', 'pwls',
       'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler',
       'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
       'subreddit_type', 'thumbnail', 'title', 'total_awards_rece

In [12]:
df[['subreddit', 'selftext', 'title']].head()

,subreddit,selftext,title
0,boardgames,Cole confirmed in the KS updates that an expan...,Oath expansion confirmed!
1,boardgames,I’m planning on buying 3 games mostly because ...,Thoughts about these games
2,boardgames,Just played my first game of Ark Nova and was ...,Help settle an Ark Nova debate.
3,boardgames,,Getting ready for a vendor event in a few week...
4,boardgames,,"Been missing party games lately, so I publishe..."


In [ ]:
# get lots of comments quickly with https://www.reddit.com/r/pushshift/comments/newjr2/best_way_to_fetch_100k_or_more_comments/
# https://www.ergosum.co/how-to-use-the-reddit-api-with-pushshift-to-bypass-api-limits/

In [18]:
#Create your timestamps and queries for your search URL
#https://www.unixtimestamp.com/index.php > Use this to create your timestamps
#after = "1577836800" #Submissions after this timestamp (1577836800 = 01 Jan 20)
#before = "1607040000" #Submissions before this timestamp (1607040000 = 04 Dec 20)
#query = "Cyberpunk 2077" #Keyword(s) to look for in submissions
sub = "Games" #Which Subreddit to search in

#subCount tracks the no. of total submissions we collect
count = 0
#subStats is the dictionary where we will store our data.
data = {}

In [ ]:
#Will need to filter out ads.

#Reddit API pulls most recent posts first. So I need to use before and go backwards. 

In [137]:
2628288*12

31539456

In [122]:
#Get today's date in UTC timestamp
from datetime import timezone
dt = datetime.datetime.now(timezone.utc)
utc_time = dt.replace(tzinfo=timezone.utc)
today = utc_time.timestamp()

print(today)    

# One year is about 31,536,000 datetime, one month is 2628288. To keep from pulling redundant posts, I'll set a loop that will increase my 'before' field by a month.
# It doesn't really matter that this is exact. These subreddits are active, so a one-month span works fine.

onemonth = 2628288


# Write a loop that creates a dictionary of before:after values to pass into my api function

rangelist={}
for i in range (0, 10):
    after = int(round(today - (onemonth*(i+1)),0)) 
    before = int(round(today - (onemonth*i), 0)) #When i is 0, "before" is set to today.
    rangelist[before]=after
rangelist

1651109255.083661


{1651109255: 1648480967,
 1648480967: 1645852679,
 1645852679: 1643224391,
 1643224391: 1640596103,
 1640596103: 1637967815,
 1637967815: 1635339527,
 1635339527: 1632711239,
 1632711239: 1630082951,
 1630082951: 1627454663,
 1627454663: 1624826375}

In [125]:
# Choosing the fields I want to pull from the API
fields = ['created_utc', 'title', 'selftext', 'author', 'upvote_ratio',  'is_created_from_ads_ui', 'url', 'crosspost_parent_list', 'total_awards_received']

def query(subreddit, num, before, after):   
    params = {
    'subreddit': subreddit,
    'size':num,
    'before':before,
    'after':after,
    'fields':fields}    # defined above
    url = 'https://api.pushshift.io/reddit/search/submission/?'
    res = requests.get(url, params)

    data = res.json()
    test = pd.DataFrame(data['data'])
    return(test)

# Testing my function
query('learnpython', 1, 1651108665, 1648480377)

,author,created_utc,is_created_from_ads_ui,selftext,title,total_awards_received,upvote_ratio,url
0,unit111,1648482436,False,Basically title. I am trying to figure out if ...,Has anybody used Playwright with Behave,0,1.0,https://www.reddit.com/r/learnpython/comments/...


In [129]:
# all these parts could be consolidated into a single function probably. how much does it mtter?

#Pull and make dataframe for learnpython sub
listdfs=[query('learnpython', 100, key, val) for key, val in rangelist.items()]

learnpython=pd.concat(listdfs)
learnpython.info()

did 100
did 100
did 100
did 100
did 100
did 100
did 100
did 100
did 100
did 100
<class 'pandas.core.frame.DataFrame'>
Int64Index: 999 entries, 0 to 99
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   author                  999 non-null    object 
 1   created_utc             999 non-null    int64  
 2   is_created_from_ads_ui  999 non-null    bool   
 3   selftext                999 non-null    object 
 4   title                   999 non-null    object 
 5   total_awards_received   999 non-null    int64  
 6   upvote_ratio            999 non-null    float64
 7   url                     999 non-null    object 
 8   crosspost_parent_list   1 non-null      object 
dtypes: bool(1), float64(1), int64(2), object(5)
memory usage: 71.2+ KB


In [130]:
#Pull and make dataframe for learnsql sub
listdfs=[query('learnsql', 100, key, val) for key, val in rangelist.items()]

learnsql=pd.concat(listdfs)
learnsql.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 955 entries, 0 to 98
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   author                  955 non-null    object 
 1   created_utc             955 non-null    int64  
 2   is_created_from_ads_ui  955 non-null    bool   
 3   selftext                955 non-null    object 
 4   title                   955 non-null    object 
 5   total_awards_received   955 non-null    int64  
 6   upvote_ratio            955 non-null    float64
 7   url                     955 non-null    object 
 8   crosspost_parent_list   16 non-null     object 
dtypes: bool(1), float64(1), int64(2), object(5)
memory usage: 68.1+ KB


In [121]:
allofit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 99
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   author                  200 non-null    object 
 1   created_utc             200 non-null    int64  
 2   is_created_from_ads_ui  200 non-null    bool   
 3   selftext                200 non-null    object 
 4   title                   200 non-null    object 
 5   total_awards_received   200 non-null    int64  
 6   upvote_ratio            200 non-null    float64
 7   url                     200 non-null    object 
dtypes: bool(1), float64(1), int64(2), object(4)
memory usage: 12.7+ KB


In [ ]:
# H2 Cleaning 
# - Are missing values imputed/handled appropriately?
# - Are distributions examined and described?
# - Are outliers identified and addressed?
# - Are appropriate summary statistics provided?
# - Are steps taken during data cleaning and EDA framed appropriately?
# - Does the student address whether or not they are likely to be able to answer their problem statement with the provided data given what 
# - they've discovered during EDA?

In [ ]:
# H4 imports

In [ ]:
# Imports

In [135]:
learnsql['url'].duplicated().unique #no duplicate rows in data

<bound method Series.unique of 0     False
1     False
2     False
3     False
4     False
      ...  
94    False
95    False
96    False
97    False
98    False
Name: url, Length: 955, dtype: bool>

In [136]:
learnpython['url'].duplicated().unique #no duplicate rows in data

<bound method Series.unique of 0     False
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Name: url, Length: 999, dtype: bool>

In [ ]:
# Check dtypes, string length, etc

In [138]:
learnsql.isna().sum()

author                      0
created_utc                 0
is_created_from_ads_ui      0
selftext                    0
title                       0
total_awards_received       0
upvote_ratio                0
url                         0
crosspost_parent_list     939
dtype: int64

In [139]:
learnpython.isna().sum()

author                      0
created_utc                 0
is_created_from_ads_ui      0
selftext                    0
title                       0
total_awards_received       0
upvote_ratio                0
url                         0
crosspost_parent_list     998
dtype: int64

In [ ]:
# H2 Exploratory Data Analysis

In [ ]:
# EDA

In [ ]:
#### H4 Chart title md

In [ ]:
# chart

In [ ]:
#### H4 Chart title md

In [ ]:
# chart

In [ ]:
params = {
    'subreddit':'news',
    'size':50,
    'q':'keyword'
}

In [ ]:
# H3 Export df

In [ ]:
# export df